# Netflix Movies and TV Shows Analysis

## Description

This dataset contains information concerning TV Shows and Movies added to the Netflix catalog, including:
- General information: id, title, type (TV Show or Movie), director, cast and a brief description.
- Date fields: When the show was released and when it was added to the catalog.
- Categorization: Rating and category in which the show is listed.

In the present notebook I go through some wrangling, and some exploratory analysis as well, gaining insight of the kind of
content available in the Netflix and how the nature of the content has changed (if it changed at all) through the years.

## Loading the data

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
from wordcloud import (
    STOPWORDS,
    WordCloud
)
pyo.init_notebook_mode()

In [ ]:
df = pd.read_csv("../input/netflix-shows/netflix_titles.csv")
df.head()

### Basic Information

In [ ]:
df.info()

Given that `describe` doesn't calculate the number of unique values for numerical columns, we can print this count ourselves:

In [ ]:
df.nunique()

## Data Transformation
### Missing Data Analysis

We begin our analysis by determining the number of rows with holes for each column:

In [ ]:
df.isna().sum()

As we can see, we have missing values for:
* `director`: we won't be using it, so we'll discard it.
* `cast`: we won't be using it, so we'll discard it.
* `country`: we'll fill missing values here with the `mode`.
* `date_added`: if you look closely, you'll notice that it's safe to do a `ffill`. Plus, the rows with missing values for
this column are the last in the dataset.
* `rating`: given that there are only 10 empty values. I decided to google the corresponding ratings and fill in the holes with the appropriate values.

First, I delete the columns I don't need.

In [ ]:
df.drop(['director', 'cast'], axis=1, inplace=True)

I fill the missing values for `country` with the `mode`.

In [ ]:
df['country'].fillna(df['country'].mode()[0], inplace=True)

I forward fill the holes in `date_added`, which are the last records in the dataset.

In [ ]:
df['date_added'] = df['date_added'].ffill()

To solve the issue of the missing ratings, I create a dictionary where the key is the `show_id` of the affected records,
and the values are the corresponding ratings I found after googling the shows. Then, I iterate this `dict`
filling in the values.

In [ ]:
replacements = {
    211: 'TV-MA',
    2411: 'PG-13',
    3288: 'TV-MA',
    4056: 'TV-MA',
    4402: 'TV-G',
    4403: 'TV-G',
    4706: 'TV-14',
    5015: 'TV-MA',
    5234: 'TV-MA',
    6231: 'TV-Y'
}

for show_id, rating in replacements.items():
    df.iloc[show_id, 6] = rating

At this point, the missing data issue should be resolved. We can check for missing data once more to confirm we no longer have empty values.

In [ ]:
df.isna().sum()

### Dropping additional columns

We now proceed to drop some other columns that we are not going to need for our process. I decided to drop `listed_in`,
given that it contains 461 unique values, which makes it kind of useless for any analysis. I removed `description` given
that I don't plan on any information out of it.

In [ ]:
df.drop(['listed_in', 'description'], axis=1, inplace=True)

### Using `show_id` as index

As the `show_id` column doesn't include duplicates, we can well use it as the index of the `DataFrame`.

In [ ]:
df.set_index('show_id', inplace=True)

### Duplicated Data Analysis

Let's check for duplicated rows:

In [ ]:
df[df.duplicated()]

As we can see, there's a duplicated record. We can print the duplicates just to corroborate that nothing
odd is going on:

In [ ]:
df[df['title'] == 'Sarkar']

Having checked that both rows are actually identical (except for the `show_id`, obviously), we can delete
the duplicated row.

In [ ]:
df.drop_duplicates(inplace=True)

### Enforcing data types

Checking the data types, we can see that there's some work to do:
- `date_added` is not a proper `datetime`.
- `type` can be coverted to a `Categorical` type.

In [ ]:
df.dtypes

#### Converting values for `date_added` to Dates

If you take a look at the content of the `date_added` column you'll see that it contains dates in a variety
of formats. Lucky for us, `Pandas` can deal create Datetime objects out of all of them:

In [ ]:
df['date_added'] = pd.to_datetime(df['date_added'])

#### Converting `type` into a Categorical type

This column contains two possible values. Although it's not really needed for our analysis, we can transform this column
into a categorical one.

In [ ]:
df['type'] = pd.Categorical(df['type'])

### Working with `country` (segregating american content)

If we explore the `country` column, we'll see that it has 544 different values:

In [ ]:
df['country'].nunique()

We will also notice that roughly half of the rows contain the string "United States" in `country`.

In [ ]:
len(df[df['country'].str.contains('United States')].index)

Therefore, we can add a column, called `american`, to flag content produced, at least partially, by America.

In [ ]:
df['american'] = df.apply(lambda row: 'United States' in row['country'], axis=1).replace({True: 'Yes', False: 'No'})

### Extracting `month` and `year` from `date_added`

Having parsed `date_added` to Datetime, we can create additional columns for the `month` and `year` to allow for additional
analysis in a convenient way.

In [ ]:
df['month_added'] = pd.DatetimeIndex(df['date_added']).month
df['year_added'] = pd.DatetimeIndex(df['date_added']).year

### Working with `rating`

If we take a look at the `ratings` column, we'll see that it contains 14 different values. They are:

In [ ]:
df['rating'].unique()

As you may know, these _maturity ratings_ are earned based on the kind of content of the show, and they stipulate the recommended age for viewers.
Based on the information found on the [Amazon's Help & Customer Service](https://www.amazon.com/gp/help/customer/display.html?nodeId=G2C2CPZWGZWHZ42J) site,
we can map ratings like this:

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Rating</th>
      <th>Age Restriction</th>
    </tr>
  </thead>
  <tbody>
<tr><td>TV-PG</td><td>7</td></tr>
<tr><td>TV-MA</td><td>18</td></tr>
<tr><td>TV-Y7-FV</td><td>7</td></tr>
<tr><td>TV-Y7</td><td>7</td></tr>
<tr><td>TV-14</td><td>16</td></tr>
<tr><td>R</td><td>18</td></tr>
<tr><td>TV-Y</td><td>0</td></tr>
<tr><td>NR</td><td>18</td></tr>
<tr><td>PG-13</td><td>13</td></tr>
<tr><td>TV-G</td><td>0</td></tr>
<tr><td>PG</td><td>7</td></tr>
<tr><td>G</td><td>0</td></tr>
<tr><td>UR</td><td>18</td></tr>
<tr><td>NC-17</td><td>18</td></tr>
  </tbody>
</table>
</div>

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Name</th>
      <th>Age Restriction</th>
    </tr>
  </thead>
  <tbody>
<tr><td>Kids</td><td>0</td></tr>
<tr><td>Older Kids</td><td>7</td></tr>
<tr><td>Teens</td><td>13</td></tr>
<tr><td>Young Adults</td><td>16</td></tr>
<tr><td>Adults</td><td>18</td></tr>
  </tbody>
</table>
</div>

Thus, we can create additional `Series` for this information.

In [ ]:
ratings_data = { 'TV-PG': 7,'TV-MA': 18,'TV-Y7-FV': 7,'TV-Y7': 7,
            'TV-14': 16,'R': 18,'TV-Y': 0,'NR': 18,'PG-13': 13,
            'TV-G': 0,'PG': 7,'G': 0,'UR': 18,'NC-17': 18}

ratings = pd.Series(ratings_data, name='age')
ratings

In [ ]:
ages_data = {0: 'Kids', 7: 'Older Kids', 13: 'Teens', 16: 'Young Adults', 18: 'Adults'}
ages = pd.Series(ages_data, name='description')
ages

We can now proceed to join both `Series` to put together a temporary `DataFrame` -- that I will eventually merge into
the main `DataFrame`. I also convert the columns to Categorical
because why not.

In [ ]:
ratings_df = pd.merge(ratings, ages, left_on='age', right_index=True)
age_bins = [-1, 17, 19]
ratings_df['18+'] = pd.cut(ratings_df['age'], age_bins, labels=['No', 'Yes'])

ratings_df['description'] = pd.Categorical(ratings_df['description'],
                                           categories=['Kids', 'Older Kids', 'Teens', 'Young Adults', 'Adults'],
                                           ordered=True)
ratings_df['age'] = pd.Categorical(ratings_df['age'], ordered=True)

ratings_df

You probably noticed that I created one extra column: `18+`. The idea is to flag the content for adults.

We now merge the additional data we generated for ratings into the main `DataFrame`. I will also rename the new columns with more appropriate names.

In [ ]:
df = pd.merge(df, ratings_df, left_on='rating', right_index=True)
df.rename(columns={'age': 'recommended_age', 'description': 'rating_description'}, inplace=True)
df.head()

## Descriptive Statistics

Having dropped columns we don't need, added some extra columns, enforced data types and removed duplicates,
our data is finally ready to start our analysis. Let's begin by printing some descriptive analytics.

### Statistics for Numerical Variables

In [ ]:
df.describe().T

### Statistic for Categorical Variables

In [ ]:
df.describe(include=[object, 'category']).T

### Types

In [ ]:
df.dtypes

In [ ]:
def group_and_count_by(data, column_names:list, reset_index:bool = True) -> pd.DataFrame:
    _df = data.filter(column_names).reset_index().groupby(column_names).count().rename(columns={'show_id': 'count'})
    if reset_index:
        _df = _df.reset_index()
    return _df.sort_values(by=['count'], ascending=False)


def plot_pie(data, fig, row, col, top:int = -1):
    labels = data['labels']
    values = data['values']
    if top > 0:
        labels = data['labels'][:top]
        labels.loc[labels.index.max() + 1] = "Others"
        values = data['values'][:top]
        values.loc[values.index.max() + 1] = data['values'][top:].sum()
    fig.add_trace(
        go.Pie(labels=labels,
               values=values,
               name=data['name']),
        row, col
    )

def plot_pie_pair(data_a, data_b, title, top:int = -1):
    fig = make_subplots(rows=1, cols=2, specs=[[{"type": "domain"}, {"type": "domain"}]])
    plot_pie(data_a, fig, 1, 1, top)
    plot_pie(data_b, fig, 1, 2, top)
    fig.update_traces(hole=.4, hoverinfo="label+percent+name")
    fig.update_layout(
        title_text=title,
        annotations=[
            dict(text=data_a['name'], x=0.16
                 , y=0.5, font_size=12, showarrow=False),
            dict(text=data_b['name'], x=0.82, y=0.5, font_size=12, showarrow=False)
        ])
    fig.show()

## Releases by type throughout the years

I plot a bar char showing the number of releases through the years. This allows us to see how recent the
content is.

In [ ]:
releases_by_year = group_and_count_by(df, (['release_year', 'type']))

fig = px.bar(releases_by_year, x='release_year', y='count', color='type')
fig.show()

As we can see, almost all the content was released in the last 10 years. We can support this plot by
looking at the quartiles printed before. There, we can see that only 25% of the content was release before 2013. We
can also note that 25% of the content is less than 2-year old (released on 2018 or later).

Another takeaway from this plot is that the number of TV-Shows has increased significantly, accounting for roughly
30% ~ 40% of the content released the last 3 years. Moreover, we can plot a line chart showing this:

In [ ]:
fig = px.line(releases_by_year.sort_values(by=['release_year']), x="release_year", y="count", color='type')
fig.show()

## Shows by Rating

Let's now study how the TV Shows and Movies are spread across different ratings. Now, working with the `rating` column is
not really that practical for most visualizations, given that it has 14 different values. Instead, we can use the
rating categories I created before. Thus, we can see how content is divided among age ranges.


In [ ]:
shows_by_rating = group_and_count_by(df, ['rating_description', 'type'])
fig = px.bar(shows_by_rating, x='rating_description', y='count', color='type', barmode='group')
fig.show()

From the chart we can see that content is more oriented toward adult audiences. This is specially true for movies. In the
following pie charts this proportion is made even clearer.

In [ ]:
ratings_tv_shows = shows_by_rating[shows_by_rating['type'] == 'TV Show']
ratings_movies = shows_by_rating[shows_by_rating['type'] == 'Movie']
data_a = {
    'labels': ratings_tv_shows['rating_description'],
    'values': ratings_tv_shows['count'],
    'name': "TV Shows"
}

data_b = {
    'labels': ratings_movies['rating_description'],
    'values': ratings_movies['count'],
    'name': "Movies"
}

plot_pie_pair(data_a, data_b, 'TV Shows and Movies')

We can plot the same data aggregated by the true `rating`. However, this produces a bunch of small sectors that don't
add much meaning to the visualization. Therefore, it's best to establish a number of representative ratings and group the
rest under "Others".

In [ ]:
shows_by_rating = group_and_count_by(df, ['rating', 'type'])
ratings_tv_shows = shows_by_rating[shows_by_rating['type'] == 'TV Show']
ratings_movies = shows_by_rating[shows_by_rating['type'] == 'Movie']

data_a = {
    'labels': ratings_tv_shows['rating'],
    'values': ratings_tv_shows['count'],
    'name': "TV Shows"
}

data_b = {
    'labels': ratings_movies['rating'],
    'values': ratings_movies['count'],
    'name': "Movies"
}

plot_pie_pair(data_a, data_b, 'TV Shows and Movies', 5)

Analyzing the previous chart, we can see that the two predominant ratings are [TV-MA](https://rating-system.fandom.com/wiki/TV-MA)
and [TV-14](https://rating-system.fandom.com/wiki/TV-14), which are assigned to shows with strong language and sexual content.

Let's wrap up the study of ratings by focusing exclusively on content for adults. For this, I create a pivot table as a
 quick and easy way to get this information. Then, as an alternative to bar charts, I use a heatmap for plotting the data.

In [ ]:
shows_18 = group_and_count_by(df, ['type', '18+'])
pivot_18 = pd.pivot_table(shows_18, values='count', index=['type'], columns=['18+'])
pivot_18

This table gives us valuable information: the number of movies for adults only is almost half of the total. On the other
hand, there number of TV Shows for adults is about a third of the total.

In [ ]:
fig = px.imshow(pivot_18, color_continuous_scale='Agsunset')
# fig = px.bar(shows_18, x='18+', y='count', color='type', barmode='group')
fig.show()


## Catalog updates through the years

We can also study how the catalog has been updated. I use line charts throughout this section, given that is simple and
easy to understand.

### TV Shows and Movies

I start by plotting two series showing the number of shows added by type through the years. In both cases we can see that
starting on 2016 the amount of content added has increased drastically. Another takeaway is that the number of shows added
the last years seems to be half the number of movies added.

In [ ]:
catalog_updates_by_type = group_and_count_by(df, ['year_added', 'type'])
fig = px.line(catalog_updates_by_type.sort_values(by=['year_added']), x="year_added", y="count", color='type')
fig.show()

### American vs International content

I now plot the rate Netflix has been added american content and content filmed abroad. From this chart we learn that the
content seems to be evenly split and added at about the same rate.

In [ ]:
catalog_updates_by_american = group_and_count_by(df, ['year_added', 'american'])
fig = px.line(catalog_updates_by_american.sort_values(by=['year_added']), x="year_added", y="count", color='american')
fig.show()

### 18+ content

I apply the same criterion to compare content for adults only against those for kids and teenagers.

In [ ]:
catalog_updates_by_type = group_and_count_by(df, ['year_added', '18+'])
fig = px.line(catalog_updates_by_type.sort_values(by=['year_added']), x="year_added", y="count", color='18+')
fig.show()

### Totals by month

By aggregating the number of shows added each month, we can determine if there are any months that see more content added
than other.

In [ ]:
shows_by_month = group_and_count_by(df[df['year_added'] < 2020], ['month_added', 'type'])
fig = px.bar(shows_by_month, x='month_added', y='count', color='type', barmode='group')
fig.show()


In fact, we see that the last three have the highest numbers of shows added. This is to be expected, given that in the
winter months people spend more time at home. So, it makes sense for Netflix to update the catalog in the months before the
holidays.

## Word Cloud for `title`

Using `WordCloud` we can obtain the words most frequently used in titles.

In [ ]:
wordcloud = WordCloud(
    height = 1000,
    width = 1500,
    background_color = 'black',
    stopwords = STOPWORDS).generate(str(' '.join(df['title'].values)))
fig = plt.figure(
    figsize = (30, 20),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

## Movies duration

Lastly, I'll study the duration of the american and foreign movies. First, I generate a box plot.


In [ ]:
movies_duration = df[df['type'] == 'Movie'].filter(['american', 'duration'])
movies_duration['duration'] = movies_duration['duration'].map(lambda x: x.replace(' min', ''))
movies_duration['duration'] = pd.to_numeric(movies_duration['duration'])

fig = px.box(movies_duration, y="duration", color='american')
fig.show()


This tells us that,
in general, foreign movies are longer than american productions. Also, the american movies seem to be more concentrated around
the median than the foreign. We can further cement this insight by means of a violin plot.

In [ ]:
fig = px.violin(movies_duration, y="duration", color="american", box=True, hover_data=movies_duration.columns)
fig.show()